In [ ]:
%pip install -q -U py3dti miniaudio soundcard

In [ ]:
from math import sin, cos, radians
from IPython.display import Audio
import soundcard
import py3dti

## Setup py3dti

In [ ]:
sample_rate = 44100
block_size = 512
file_path = f'../3dti_AudioToolkit/resources/AudioSamples/Anechoic Speech {sample_rate}.wav'

In [ ]:
renderer = py3dti.BinauralRenderer(sample_rate, block_size, resampled_angular_resolution=5)

### Add listener

In [ ]:
listener = renderer.add_listener()
listener.load_hrtf_from_sofa(f'../3dti_AudioToolkit/resources/HRTF/SOFA/3DTI_HRTF_IRC1008_512s_{sample_rate}Hz.sofa')
# listener.load_hrtf_from_3dti(f'3dti_AudioToolkit/resources/HRTF/SOFA/3DTI_HRTF_IRC1008_512s_{sample_rate}Hz.3dti-hrtf') # faster, but less common file format
listener.load_ild_near_field_effect_table(f'../3dti_AudioToolkit/resources/ILD/NearFieldCompensation_ILD_{sample_rate}.3dti-ild')

In [ ]:
listener.position = (0, 0, 0) # default position
listener.orientation = (1, 0, 0, 0) # default orientation

In [ ]:
listener.position, listener.orientation

### Add environment

In [ ]:
environment = renderer.add_environment()
environment.load_brir_from_sofa(f'../3dti_AudioToolkit/resources/BRIR/SOFA/3DTI_BRIR_medium_{sample_rate}Hz.sofa')
# environment.load_brir_from_3dti(f'3dti_AudioToolkit/resources/BRIR/SOFA/3DTI_BRIR_medium_{sample_rate}Hz.3dti-brir') # faster, but less common file format

### Add source(s)

In [ ]:
source = renderer.add_source()

In [ ]:
source.position = (2, 2, 2)
source.orientation = (2, 1, 1, 1)

## Read source samples

In [ ]:
from miniaudio import decode_file, SampleFormat
import numpy as np
decoded_file = decode_file(filename=file_path, output_format=SampleFormat.FLOAT32,
                           nchannels=1, sample_rate=sample_rate)
samples = np.asarray(decoded_file.samples)

## Rendering of static listener and sources

In [ ]:
def render_static(renderer, sources):
    binaural_length = np.max(list(map(len, sources.values())))
    binaural_output = np.zeros((2, binaural_length), dtype=np.float32)
    for begin in np.arange(0, binaural_length - renderer.buffer_size, renderer.buffer_size):
        end = begin + renderer.buffer_size
        for source, samples in sources.items():
            left_buffer, right_buffer = source.process_anechoic(samples[begin:end])
            binaural_output[0, begin:end] += left_buffer
            binaural_output[1, begin:end] += right_buffer

        left_buffer, right_buffer = environment.process_virtual_ambisonic_reverb()
        binaural_output[0, begin:end] += left_buffer
        binaural_output[1, begin:end] += right_buffer
    return binaural_output

In [ ]:
sources = {source: samples}
binaural_samples = render_static(renderer, sources)
Audio(binaural_samples, rate=sample_rate)

## Blockwise processing accumulated into binaural np.array

In [ ]:
binaural_output = np.zeros((2, len(samples)), dtype=np.float32)
for start in np.arange(0, len(samples) - block_size, block_size):
    input_buffer = samples[start:start+block_size]
    # Comment the next line to keep the source stationary in the location defined above,
    # otherwise it will circle counter-clockwise in the frontal plane
    source.position = (0, sin(radians(start/block_size)), cos(radians(start/block_size)))
    left_buffer, right_buffer = source.process_anechoic(input_buffer)
    binaural_output[0, start:start+block_size] = left_buffer
    binaural_output[1, start:start+block_size] = right_buffer

    left_buffer, right_buffer = environment.process_virtual_ambisonic_reverb()
    binaural_output[0, start:start+block_size] += left_buffer
    binaural_output[1, start:start+block_size] += right_buffer

In [ ]:
Audio(binaural_output, rate=sample_rate)

## Real-Time Output

In [ ]:
with soundcard.default_speaker().player(samplerate=sample_rate, channels=2) as stereo_speaker:
    for start in np.arange(0, len(samples) - block_size, block_size):
        input_buffer = samples[start:start+block_size]
        # Comment the next line to keep the source stationary in the location defined above,
        # otherwise it will circle counter-clockwise in the horizontal plane
        source.position = (cos(radians(start/block_size)), sin(radians(start/block_size)), 0)
        stereo_output = np.column_stack(source.process_anechoic(input_buffer))
        stereo_output += np.column_stack(environment.process_virtual_ambisonic_reverb())
        stereo_speaker.play(stereo_output)

## Some more (modifiable) properties with sensible default values

In [ ]:
listener.head_radius, listener.ild_attenuation

In [ ]:
source.spatialization_mode, source.anechoic_processing, source.reverb_processing

In [ ]:
source.far_distance_effect, source.near_field_effect

In [ ]:
source.propagation_delay, source.anechoic_distance_attenuation, source.anechoic_distance_attenuation_smoothing